In [251]:
from math import inf
from deuxpots.box import load_box_mapping
from deuxpots import CERFA_VARIABLES_PATH
from deuxpots.tax_calculator import IncomeSheet, compute_tax

In [238]:
BOX_MAPPING = load_box_mapping(CERFA_VARIABLES_PATH)
FLOAT_BOX_CODES = [box.code for box in BOX_MAPPING.values() if box.reference.type == "float"]
FLOAT_BOX_CODES

['0CF', '0CG', '0CH', '0CI', '0CR', '0DJ', '0DN']

In [252]:
RATE_PIECES = [
    (0, 10777, 0),
    (10778, 27478, 0.11),
    (27479, 78570, 0.3),
    (78571, 168994, 0.41),
    (168994, inf, 0.45),
]

def apply_piecewise_rate(income):
    return sum(
        (min(upper, income) - min(lower, income)) * rate
        for lower, upper, rate in RATE_PIECES)

def apply_inverse_piecewise_rate(tax):
    income = 0
    explained_tax = 0
    for lower, upper, rate in RATE_PIECES:
        new_explained_tax = (upper - lower) * rate
        if new_explained_tax < tax:
            income += upper - lower
            explained_tax += new_explained_tax
        else:
            income += (tax - explained_tax) / rate
            break
    return income

In [253]:
apply_piecewise_rate(36000)

4393.299999999999

In [241]:
apply_inverse_piecewise_rate(25950.19)

99997.0

In [242]:
apply_inverse_piecewise_rate(0)

ZeroDivisionError: division by zero

In [288]:
sheets = [
    IncomeSheet({
        'pre_situation_famille': 'C',
        'pre_situation_residence': 'M',
        '0DA': 1950,
        '0CF': 1,
        '0CG': 1,
        '1AJ': 60000,
    }),
    IncomeSheet({
        'pre_situation_famille': 'C',
        'pre_situation_residence': 'M',
        '0DA': 1950,
        '0CH': 1,
        '0CI': 1,
        '1AJ': 60000,
    }),
]

In [289]:
results = []
for sheet in sheets:
    result = compute_tax(sheet)
    print(
        result.total_tax,
    )
    results.append(result)

4765
7279


In [290]:
results

[SimulatorResult(total_tax=4765, already_paid=0, remains_to_pay=4765, household_size=2.0, taxable_income=54000),
 SimulatorResult(total_tax=7279, already_paid=0, remains_to_pay=7279, household_size=1.5, taxable_income=54000)]

In [274]:
int(round(1.265 * apply_piecewise_rate(result.taxable_income / 1.265)))

8096

In [267]:
apply_piecewise_rate(48110)

8026.3

In [69]:
8026

8026

In [70]:
(48110-53700) / 53700

-0.10409683426443203